In [ ]:
import re
import random

Questions = "Q.txt"
Answers = "A.txt"

with open(Questions, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')
with open(Answers, 'r', encoding='utf-8') as f:
  lines2 = f.read().split('\n')

#Preprocess
regex_patterns = {
            r"i'm": "i am",
            r"he's": "he is",
            r"she's": "she is",
            r"that's": "that is",
            r"what's": "what is",
            r"where's": "where is",
            r"how's": "how is",
            r"'till": "until",
            r"\'ll": " will",
            r"\'ve": " have",
            r"\'re": " are",
            r"\'d": " would",
            r"won't": "will not",
            r"can't": "cannot",
            r"didn't": "did not",
            r"[^\w\s]": "",
        }

for i in range(len(list(regex_patterns.keys()))):
  da_key = list(regex_patterns.keys())[i]
  da_value = list(regex_patterns.values())[i]
  lines = [re.sub(da_key, da_value, line.lower(), flags=re.IGNORECASE).strip() for line in lines]


for i in range(len(list(regex_patterns.keys()))):
  da_key = list(regex_patterns.keys())[i]
  da_value = list(regex_patterns.values())[i]
  lines2 = [re.sub(da_key, da_value, line.lower(), flags=re.IGNORECASE).strip() for line in lines2]

pairs = list(zip(lines,lines2))
print(pairs[1295])

('what are the evolving dangers of public wifi networks', 'public wifi use is convenient but risky due to cybersecurity threats like maninthemiddle attacks unencrypted networks malware distribution snooping and malicious hotspots')


In [ ]:
import numpy as np

input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()
for line in pairs[:1295]:
  input_doc, target_doc = line[0], line[1]
  input_docs.append(input_doc)
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  target_doc = '<BOS> ' + target_doc + ' <EOS>'
  target_docs.append(target_doc)
  #print(input_doc, target_doc)
  # Word2vec için sözlük tokeni -vvv-
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)

input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())


max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.

    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.
        if timestep > 0:
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

In [ ]:
print(pairs[:64])

[('how would your parents describe what you do for a living', 'parents see me as a travel enthusiast focusing on delta sky miles and marriott rewards though aware i work in security they lack techsavviness'), ('thats funny because of course this year at least those frequentflyer miles and hotel points have not been really racking up the way they usually do im sure', 'challenging year thrilled with delta and marriott status extension misses traveling but appreciates being at home for the past six to seven months making the best of a difficult situation'), ('how did you happen to develop expertise in those areas', 'acquisitions expanded the portfolio now specializing in retail and telecom security in telecom started with cellular networks and telephony 810 years ago primarily works in retail and telecom enjoying both industries'), ('i would imagine as in most industries having specific knowledge of security issues in retail or in telecommunications is beneficial just because of the uniqu

In [ ]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model

#keras.utils.set_random_seed(812)

dimensionality = 256

batch_size = 16
epochs = 900

#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]

#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
if True:
  #Model
  training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
  #Compiling
  #training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
  training_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
  #Training

  training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
  training_model.save('training_model.h5')

Epoch 1/900
65/65 [==============================] - 8s 52ms/step - loss: 2.4774 - accuracy: 0.0352 - val_loss: 2.1211 - val_accuracy: 0.0334
Epoch 2/900
65/65 [==============================] - 2s 25ms/step - loss: 2.2803 - accuracy: 0.0391 - val_loss: 2.1099 - val_accuracy: 0.0412
Epoch 3/900
65/65 [==============================] - 2s 25ms/step - loss: 2.2528 - accuracy: 0.0405 - val_loss: 2.1136 - val_accuracy: 0.0327
Epoch 4/900
65/65 [==============================] - 2s 29ms/step - loss: 2.2369 - accuracy: 0.0410 - val_loss: 2.1105 - val_accuracy: 0.0315
Epoch 5/900
65/65 [==============================] - 2s 31ms/step - loss: 2.2240 - accuracy: 0.0421 - val_loss: 2.1125 - val_accuracy: 0.0318
Epoch 6/900
65/65 [==============================] - 2s 24ms/step - loss: 2.2162 - accuracy: 0.0428 - val_loss: 2.1148 - val_accuracy: 0.0332
Epoch 7/900
65/65 [==============================] - 2s 24ms/step - loss: 2.2088 - accuracy: 0.0428 - val_loss: 2.1135 - val_accuracy: 0.0330
Epoch 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.models import load_model
training_model = load_model('training_model.h5')
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

latent_dim = 256
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def decode_response(test_input):
    #Getting the output states to pass into the decoder
    states_value = encoder_model.predict(test_input)
    #Generating empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_features_dict['<BOS>']] = 1.

    decoded_sentence = ''

    stop_condition = False
    while not stop_condition:
          output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)
          #Choosing the one with highest probability
          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_token = reverse_target_features_dict[sampled_token_index]
          decoded_sentence += " " + sampled_token
          #Durma kondisyonu
          if (sampled_token == '<EOS>'):
            stop_condition = True
          #Update

          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.
          states_value = [hidden_state, cell_state]

    return decoded_sentence

In [ ]:
class ChatBot:
  negative_responses = ("no", "nope", "nah")
  exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "adios")

  def start_chat(self):
    user_response = input("Hi, I'm a chatbot trained on cybersecurity. Would you like to ask me anything?\n")
    if user_response in self.negative_responses:
      print("Ok, have a great day!")
      return
    self.chat(user_response)

  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply)+"\n")

  def string_to_matrix(self, user_input):    #<-------------------- Kullanıcının girdisini matrix'e çeviriyor
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens),
      dtype='float32')
    for timestep, token in enumerate(tokens):
      if token in input_features_dict:
        user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix

  def generate_response(self, user_input):
    input_matrix = self.string_to_matrix(user_input)
    chatbot_response = decode_response(input_matrix)
    chatbot_response = chatbot_response.replace("<BOS>",'')
    chatbot_response = chatbot_response.replace("<EOS>",'')
    chatbot_response = chatbot_response.strip().capitalize() + "."
    return chatbot_response

  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Ok, have a great day!")
        return True
    return False

chatbot = ChatBot()
chatbot.start_chat()

Hi, I'm a chatbot trained on cybersecurity. Would you like to ask me anything?
What is a computer worm
1/1 [==============================] - 0s 74ms/step
A worm is a standalone malware computer program that replicates itself to spread to other.
what is public cryptography?
1/1 [==============================] - 0s 19ms/step
They improves security by firewalls an unreadable format granular security and.
What is ethical hacking? 
1/1 [==============================] - 0s 19ms/step
Directories hacking involves legally breaking into computers and devices to the cybersecurity security improvement.
What is common sertificant in ethical hacking?
1/1 [==============================] - 0s 19ms/step


KeyboardInterrupt: Interrupted by user

In [ ]:
chatbot.start_chat()
